In [ ]:
#install relevant libraries
!pip install openpyxl
!pip install geojson
!pip install mapillary

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import requests
import pandas as pd
# import geojson
import mapillary.interface as mly
import numpy as np
import folium
import math

In [ ]:
#client token
client_token = 'MLY|5195767363825644|814efd8bf99a08986aa1e1e0aeacfb18'

In [ ]:
#Set Token for using mapillary library
mly.set_access_token('MLY|5188271894567264|1a74fb89c68d98176fc9c7e61b27cd04')

{'token': 'SUCCESS'}

In [ ]:
#Specify bounding box
x_dist = 0.0025
y_dist = 0.0025
stestpoint=[48.249, 16.295]
testpoint = []
testpoint2 = []
bounds2 = []
testpoint.append(stestpoint)

#x:16
#y:48 decreasing to go down

bounds = []
for i in range(1,19): #make rows
  testpoint.append([stestpoint[0]-y_dist*i*2, stestpoint[1]])

for j in range(len(testpoint)):
  for k in range(0,29): #make columns
    testpoint2.append([testpoint[j][0],testpoint[j][1]+y_dist*k*2])
for l in range(len(testpoint2)):
  bounds2.append([(testpoint2[l][0]-x_dist, testpoint2[l][1]-y_dist), (testpoint2[l][0]+x_dist, testpoint2[l][1]+y_dist)])

In [ ]:
bounds2

[[(48.246500000000005, 16.2925), (48.2515, 16.297500000000003)],
 [(48.246500000000005, 16.2975), (48.2515, 16.302500000000002)],
 [(48.246500000000005, 16.302500000000002), (48.2515, 16.307500000000005)],
 [(48.246500000000005, 16.3075), (48.2515, 16.312500000000004)],
 [(48.246500000000005, 16.3125), (48.2515, 16.317500000000003)],
 [(48.246500000000005, 16.3175), (48.2515, 16.3225)],
 [(48.246500000000005, 16.3225), (48.2515, 16.327500000000004)],
 [(48.246500000000005, 16.3275), (48.2515, 16.332500000000003)],
 [(48.246500000000005, 16.3325), (48.2515, 16.337500000000002)],
 [(48.246500000000005, 16.337500000000002), (48.2515, 16.342500000000005)],
 [(48.246500000000005, 16.3425), (48.2515, 16.347500000000004)],
 [(48.246500000000005, 16.3475), (48.2515, 16.352500000000003)],
 [(48.246500000000005, 16.3525), (48.2515, 16.3575)],
 [(48.246500000000005, 16.3575), (48.2515, 16.362500000000004)],
 [(48.246500000000005, 16.3625), (48.2515, 16.367500000000003)],
 [(48.246500000000005, 16

In [ ]:
#View on map
map = folium.Map(location=[np.mean(stestpoint[0]), np.mean(stestpoint[1])], tiles='stamentoner', zoom_start=12, width = 1200, height = 1200)
for i in range(len(testpoint2)):
  folium.Marker(testpoint2[i], icon=folium.DivIcon(html=f"""<div style=color: blue; font-size: 1>{i}</div>""")).add_to(map)
  folium.Rectangle(bounds2[i], color="blue", weight=1, stroke=True).add_to(map)
map

In [ ]:
numFound = []
imageID = []
imageURL = []
imageLat = []
imageLong = []
imageDetectIDs = []
bbID = []

In [ ]:
#format request url and make request
#3:20 for 36 boxes


headers= { "Authorization" : "OAuth {}".format(client_token) }
metadata_endpoint = "https://graph.mapillary.com"

for i in range(431,len(bounds2)):
  url_imagesearch = metadata_endpoint+'/images?fields=id&bbox= {},{},{},{}'.format(testpoint2[i][1]-x_dist, testpoint2[i][0]-y_dist, testpoint2[i][1]+x_dist, testpoint2[i][0]+y_dist)
  response_imagesearch = requests.get(url_imagesearch, headers=headers)
  data_imageSearch = response_imagesearch.json() #convert response to json format
  print("Images found:" + str(len(data_imageSearch['data'])))  
  numFound.append(len(data_imageSearch['data']))
  print(i)
  
  iters = math.ceil(len(data_imageSearch['data'])/50)

  if len(data_imageSearch['data'])>0:
    for j in range(0,len(data_imageSearch['data']),iters):
      url_image = metadata_endpoint+'/{}?fields=id,thumb_1024_url,computed_geometry'.format(data_imageSearch['data'][j]['id'])
      response_image = requests.get(url_image, headers=headers)
      data_image = response_image.json()
      imageID.append(data_image['id'])
      imageURL.append(data_image['thumb_1024_url'])
      try:
        imageLat.append(data_image['computed_geometry']['coordinates'][0])
        imageLong.append(data_image['computed_geometry']['coordinates'][1])
      except Exception: 
        imageLat.append('NA')
        imageLong.append('NA')
        print('e')
      bbID.append(i)


In [ ]:
len(numFound)

In [ ]:
#make dataframe and write to excel
data = {'image id':imageID,
        'URL':imageURL,
        'Lat': imageLat,
        'Long': imageLong,
        'Bounding Box ID':  bbID}

df = pd.DataFrame(data)
name = 'ViennaImageData2.xlsx'
df.to_excel(name)

In [ ]:
data = {'Found': numFound}

df = pd.DataFrame(data)
name = 'numFound2.xlsx'
df.to_excel(name)

In [ ]:
#read excel files
data = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/Studio/GetMapillaryImages/AllViennaImageData.xlsx')

NameError: ignored

In [ ]:
len(data)

24555

In [ ]:
save_location = '/content/drive/MyDrive/Colab Notebooks/Studio/GetMapillaryImages/'

for i in range(0,len(data)):
  img_data = requests.get(data['URL'][i]).content
  image_Name = str(data['image id'][i])+'.png'
  name = save_location + image_Name 
  with open(name, 'wb') as handler:
    handler.write(img_data)

In [ ]:
#images found per bounding box
data = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/Studio/GetMapillaryImages/Found.xlsx')

In [ ]:
#View on map
map = folium.Map(location=[np.mean(stestpoint[0]), np.mean(stestpoint[1])], zoom_start=12, width = 1200, height = 1200)

for i in range(len(testpoint2)):
  folium.Marker(testpoint2[i], icon=folium.DivIcon(html=f"""<div style=color: blue; font-size: 1>{i}</div>""")).add_to(map)
  folium.Rectangle(bounds2[i], color="red", weight=1, stroke=True).add_to(map)
map